In [1]:
%load_ext autoreload
%autoreload 2
from constants import *
from evaluator import *
from model import *
from mutator import *
from prompt import *
from task import *

import warnings
warnings.filterwarnings("ignore")

In [2]:
import re

def extract_last_numeric_value(input_string):
    # Regular expression pattern to extract all numeric values with periods and ignoring commas
    pattern = r'[\d,.]+'

    # Find all matches using re.finditer()
    matches = re.finditer(pattern, input_string)

    # Initialize a variable to store the last numeric value
    last_numeric_value = None

    # Iterate through the matches and update the last_numeric_value
    for match in matches:
        numeric_value = match.group()
        # Remove commas if needed
        numeric_value = numeric_value.replace(",", "")
        last_numeric_value = numeric_value

    return last_numeric_value

In [3]:
def evaluate_func(orig, pred):
    orig_value = extract_last_numeric_value(orig)
    pred_value = extract_last_numeric_value(pred)
    try:
        return abs(float(orig_value) - float(pred_value)) < 1e-6
    except:
        return False

In [4]:
from datasets import load_dataset

# mutation_model = Model(provider="huggingface", model_name="mistralai/Mistral-7B-Instruct-v0.1")
mutation_model = Model(provider="quantized_llama", model_name="TheBloke/Mistral-7B-OpenOrca-GGUF", model_file="mistral-7b-openorca.Q4_K_M.gguf", model_type="mistral")
evaluation_model = mutation_model

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
system_instruction = ""
thinking_style = ""
task = Task(load_dataset('gsm8k', 'main'),
            'Solve the math word problem, giving your answer as an arabic numeral.',
            evaluate_func)
initial_prompt = Prompt(task.initial_prompt, system_instruction, thinking_style, None)

In [ ]:
task.test()

In [ ]:
evaluator = Evaluator()
all_pop = [initial_prompt]
scores = evaluator.evaluate(evaluation_model, task, "one", all_pop, len(task.test()))

# Print prompt and their score
prompt_scores = [(all_pop[i].get_accuracy(), all_pop[i]) for i in range(len(all_pop))]
prompt_scores = sorted(prompt_scores, key=lambda x: x[0], reverse=True)
print("Score  | Evals | Prompt")
for _, prompt in prompt_scores:
    print(f"{prompt.get_accuracy() * 100:5.2f}% | {prompt.get_num_evals():5d} | {prompt}")

# Survival of the fittest
# TODO: Create a more elaborate selection mechanism that keeps some weaker member
pop = [prompt_scores[i][1] for i in range(NUM_POPULATION)]

# Record time
end_time = time.time()
execution_time = end_time - start_time
print("Execution time: {:.2f} seconds".format(execution_time))

Evaluating test samples: [142, 387, 966, 699, 386, 1104, 1005, 743, 135, 1134, 625, 1258, 595, 1113, 619, 707, 106, 37, 13, 526, 652, 859, 1086, 1029, 740, 89, 1222, 785, 1274, 591, 54, 902, 79, 368, 885, 325, 769, 274, 50, 767, 83, 90, 872, 548, 1108, 774, 585, 738, 1107, 806, 435, 722, 264, 412, 1282, 673, 916, 1238, 691, 44, 644, 967, 1136, 369, 1036, 174, 976, 681, 1035, 539, 731, 449, 24, 965, 998, 290, 1112, 570, 517, 1052, 372, 1294, 10, 1250, 1091, 244, 974, 575, 721, 446, 203, 876, 223, 1137, 17, 912, 124, 755, 821, 314, 40, 1153, 1261, 723, 52, 925, 139, 1195, 905, 1061, 981, 262, 1109, 599, 750, 1050, 618, 711, 1290, 943, 265, 640, 18, 94, 332, 809, 228, 787, 1023, 331, 663, 163, 1289, 1127, 348, 258, 179, 1193, 949, 823, 665, 122, 1285, 857, 542, 534, 184, 1116, 173, 12, 441, 384, 573, 445, 176, 613, 1303, 1288, 467, 726, 20, 1191, 354, 280, 260, 910, 659, 729, 822, 969, 938, 1198, 759, 727, 396, 1277, 406, 922, 930, 134, 775, 920, 927, 9, 1006, 506, 411, 1098, 175, 1296, 2